In [1]:
import polars as pl
import glob
from tqdm.notebook import tqdm

DATA_PATH = '../data/proccesed_data/'

all_parquet_files = glob.glob(f"{DATA_PATH}*.parquet")

if not all_parquet_files:
    raise FileNotFoundError(f"В папке {DATA_PATH} не найдено parquet-файлов. Убедитесь, что путь указан верно.")

print(f"Найденные файлы: {all_parquet_files}")

SAMPLE_FRACTION = 0.1
lazy_frames = []

print(f"\nСоставляем план для загрузки {SAMPLE_FRACTION*100:.0f}% из каждого файла...")

for file in all_parquet_files:
    lf = pl.scan_parquet(file)
    
    total_rows = lf.select(pl.len()).collect().item()
    sample_size = int(total_rows * SAMPLE_FRACTION)
    sampled_lf = lf.with_columns(
        pl.lit(pl.arange(0, total_rows, eager=True).shuffle(seed=42)).alias("random_index")
    ).sort("random_index").head(sample_size).drop("random_index")

    lazy_frames.append(sampled_lf)
    print(f"  - Файл '{file.split('/')[-1]}': всего строк {total_rows}, берем {sample_size}")

combined_lazy_frame = pl.concat(lazy_frames)

def get_space_positions(sentence_with_spaces: str) -> list:
    """Вычисляет ground truth позиции для пробелов."""
    if not isinstance(sentence_with_spaces, str) or not sentence_with_spaces:
        return []
    words = sentence_with_spaces.split(' ')
    positions = []
    current_pos = 0
    for i, word in enumerate(words):
        current_pos += len(word)
        if i < len(words) - 1:
            positions.append(current_pos)
    return positions

print("\nНачинаем материализацию сэмпла и вычисление позиций...")
df = combined_lazy_frame.with_columns(
    pl.col("sentence_with_spaces").map_elements(
        get_space_positions, 
        return_dtype=pl.List(pl.Int64)
    ).alias("true_positions")
).collect()

df = df.sample(fraction=1, shuffle=True, seed=42)

print(f"\nВсего загружено и обработано строк (сэмпл): {len(df)}")
df.head()

Найденные файлы: ['../data/proccesed_data/item_info_train.parquet', '../data/proccesed_data/item_info.parquet', '../data/proccesed_data/pesni.parquet', '../data/proccesed_data/processed_corpus.parquet']

Составляем план для загрузки 10% из каждого файла...
  - Файл 'item_info_train.parquet': всего строк 16276308, берем 1627630
  - Файл 'item_info.parquet': всего строк 5810022, берем 581002
  - Файл 'pesni.parquet': всего строк 4940902, берем 494090
  - Файл 'processed_corpus.parquet': всего строк 9321, берем 932

Начинаем материализацию сэмпла и вычисление позиций...

Всего загружено и обработано строк (сэмпл): 2703654


id,sentence_with_spaces,sentence_without_spaces,true_positions
i64,str,str,list[i64]
1496341,"""и к черту соглашение с Янктоно…","""икчертусоглашениесЯнктоном""","[1, 2, … 18]"
1,"""Продам Камаз 6520 20 тонн""","""ПродамКамаз652020тонн""","[6, 11, … 17]"
5139734,"""Дверь газ 21""","""Дверьгаз21""","[5, 8]"
9823880,"""(937) 3-44444-8 — 10 т.""","""(937)3-44444-8—10т.""","[5, 14, … 17]"
13288818,"""Monster Tiger Nut DY229 Товар …","""MonsterTigerNutDY229 Товарновы…","[7, 12, … 26]"


In [3]:
from collections import Counter
import os
import math

from symspellpy import SymSpell


RANDOM_SEED = 42
pl.Config.set_fmt_str_lengths(100)

train_fraction = 0.9
train_size = int(train_fraction * len(df))
train_df = df.slice(0, train_size)
test_df = df.slice(train_size)

print(f"Размер обучающей выборки (train): {len(train_df)}")
print(f"Размер тестовой выборки (test): {len(test_df)}")
print("\nЯчейка 2: Данные успешно загружены и подготовлены.")

Размер обучающей выборки (train): 2433288
Размер тестовой выборки (test): 270366

Ячейка 2: Данные успешно загружены и подготовлены.


In [4]:
print("Создаем частотный словарь на обучающих данных...")
all_words_series = train_df.select(
    pl.col("sentence_with_spaces").str.split(by=" ")
).explode("sentence_with_spaces")
unigram_counts = Counter(all_words_series["sentence_with_spaces"].to_list())

calculated_max_len = all_words_series.select(
    pl.col("sentence_with_spaces").str.len_chars().max()
).item()
MAX_WORD_LEN = min(calculated_max_len, 50)
print(f"Словарь создан. Уникальных слов: {len(unigram_counts)}, макс. длина слова: {MAX_WORD_LEN}")

dictionary_path = "symspell_dictionary.txt"
print(f"Создаем временный файл словаря: '{dictionary_path}'")

with open(dictionary_path, "w", encoding="utf-8") as f:
    for word, count in unigram_counts.items():
        if word:
            f.write(f"{word} {count}\n")

sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
if not sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1):
    raise IOError("Файл словаря не может быть загружен SymSpell.")
else:
    print("Словарь успешно загружен в SymSpell.")
    
print("\nЯчейка 3: Языковая модель готова.")

Создаем частотный словарь на обучающих данных...
Словарь создан. Уникальных слов: 1919082, макс. длина слова: 50
Создаем временный файл словаря: 'symspell_dictionary.txt'
Словарь успешно загружен в SymSpell.

Ячейка 3: Языковая модель готова.


In [8]:
print("Создаем частотный словарь на обучающих данных...")
all_words_series = train_df.select(
    pl.col("sentence_with_spaces").str.split(by=" ")
).explode("sentence_with_spaces")
unigram_counts = Counter(all_words_series["sentence_with_spaces"].to_list())
calculated_max_len = all_words_series.select(
    pl.col("sentence_with_spaces").str.len_chars().max()
).item()
MAX_WORD_LEN = min(calculated_max_len, 50)
print(f"Словарь создан. Уникальных слов: {len(unigram_counts)}, макс. длина слова: {MAX_WORD_LEN}")

dictionary_path = "symspell_dictionary.txt"
print(f"Создаем временный файл словаря: '{dictionary_path}'")
with open(dictionary_path, "w", encoding="utf-8") as f:
    for word, count in unigram_counts.items():
        if word:
            f.write(f"{word} {count}\n")

NEW_PREFIX_LENGTH = 15
print(f"Инициализируем SymSpell с prefix_length={NEW_PREFIX_LENGTH}...")
sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=NEW_PREFIX_LENGTH)

if not sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1):
    raise IOError("Файл словаря не может быть загружен SymSpell.")
else:
    print("Словарь успешно загружен в SymSpell.")
    
print("\nЯчейка 3: Языковая модель готова.")

Создаем частотный словарь на обучающих данных...
Словарь создан. Уникальных слов: 1919082, макс. длина слова: 50
Создаем временный файл словаря: 'symspell_dictionary.txt'
Инициализируем SymSpell с prefix_length=15...
Словарь успешно загружен в SymSpell.

Ячейка 3: Языковая модель готова.


In [9]:
def segment_with_symspell(text: str) -> list:
    """Обертка для вызова symspellpy с защитой от OverflowError и IndexError."""
    if not text:
        return []
    try:
        result = sym_spell.word_segmentation(text, max_edit_distance=0, max_segmentation_word_length=MAX_WORD_LEN)
        words = result.segmented_string.split()
    except (OverflowError, IndexError) as e:
        print(f"\nПерехвачена ошибка ({type(e).__name__}) для строки длиной {len(text)}. Возвращаем пустой результат.")
        return []
    
    positions = []
    current_pos = 0
    for i, word in enumerate(words):
        current_pos += len(word)
        if i < len(words) - 1:
            positions.append(current_pos)
    return positions

def calculate_f1_corrected(predicted: list, true: list) -> float:
    if not predicted and not true: return 1.0
    pred_set = set(predicted)
    true_set = set(true)
    tp = len(pred_set.intersection(true_set))
    precision = tp / len(pred_set) if pred_set else 0.0
    recall = tp / len(true_set) if true_set else 0.0
    if precision + recall == 0: return 0.0
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

print("Ячейка 4: Функции для сегментации и оценки определены.")

Ячейка 4: Функции для сегментации и оценки определены.


In [10]:
print("Начинаем предсказание на тестовой выборке с помощью `symspellpy`...")

sentences_to_process = test_df["sentence_without_spaces"].to_list()

predicted_positions_list = [
    segment_with_symspell(s) 
    for s in tqdm(sentences_to_process, desc="symspellpy")
]

results_df = test_df.with_columns(
    pl.Series("predicted_positions", predicted_positions_list, dtype=pl.List(pl.Int64))
)

f1_scores = [
    calculate_f1_corrected(row["predicted_positions"], row["true_positions"])
    for row in tqdm(results_df.select(["predicted_positions", "true_positions"]).iter_rows(named=True), 
                    desc="Вычисление F1", total=len(results_df))
]

results_df = results_df.with_columns(
    pl.Series("f1_score", f1_scores, dtype=pl.Float64)
)

mean_f1 = results_df['f1_score'].mean()
print("-" * 50)
print(f"ИТОГОВЫЙ РЕЗУЛЬТАТ:")
print(f"Средний F1-score на тестовой выборке: {mean_f1:.4f}")
print("-" * 50)

os.remove(dictionary_path)
print(f"Временный файл словаря '{dictionary_path}' удален.")
print("\nЯчейка 5: Оценка завершена.")

Начинаем предсказание на тестовой выборке с помощью `symspellpy`...


symspellpy:   0%|          | 0/270366 [00:00<?, ?it/s]


Перехвачена ошибка (IndexError) для строки длиной 76. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 104. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 1. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 2. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 43. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 1. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 1. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 1. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 77. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 1. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 32. Возвращаем пустой результат.

Перехвачена ошибка (IndexError) для строки длиной 1. Возвращаем пустой результат

Вычисление F1:   0%|          | 0/270366 [00:00<?, ?it/s]

--------------------------------------------------
ИТОГОВЫЙ РЕЗУЛЬТАТ:
Средний F1-score на тестовой выборке: 0.7693
--------------------------------------------------
Временный файл словаря 'symspell_dictionary.txt' удален.

Ячейка 5: Оценка завершена.


In [11]:
print("Анализируем результаты...")

print("\nПримеры с самым низким F1-score (ошибки модели):")
print(results_df.sort("f1_score").head(5))

print("\nПримеры с самым высоким F1-score (успехи модели):")
print(results_df.sort("f1_score", descending=True).head(5))

print("\nЯчейка 6: Анализ ошибок завершен.")

Анализируем результаты...

Примеры с самым низким F1-score (ошибки модели):
shape: (5, 6)
┌──────────┬───────────────────┬───────────────────┬────────────────┬───────────────────┬──────────┐
│ id       ┆ sentence_with_spa ┆ sentence_without_ ┆ true_positions ┆ predicted_positio ┆ f1_score │
│ ---      ┆ ces               ┆ spaces            ┆ ---            ┆ ns                ┆ ---      │
│ i64      ┆ ---               ┆ ---               ┆ list[i64]      ┆ ---               ┆ f64      │
│          ┆ str               ┆ str               ┆                ┆ list[i64]         ┆          │
╞══════════╪═══════════════════╪═══════════════════╪════════════════╪═══════════════════╪══════════╡
│ 4113081  ┆ - 150 руб         ┆ -150руб           ┆ [1, 4]         ┆ []                ┆ 0.0      │
│ 4609223  ┆ Вторичка.         ┆ Вторичка.         ┆ []             ┆ [3, 6]            ┆ 0.0      │
│ 13343528 ┆ В наличии.        ┆ Вналичии.         ┆ [1]            ┆ []                ┆ 0.0      │
│